<a href="https://colab.research.google.com/github/dclark18/7331DataMiningNotebooks/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#pip install emnist

In [27]:
from __future__ import print_function
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,  Conv2D, MaxPooling2D
from keras import backend as K

In [28]:
from emnist import extract_test_samples

In [29]:
images, labels = extract_test_samples('digits')

In [30]:
images.shape

(40000, 28, 28)

In [31]:
now = datetime.datetime.now
batch_size = 128
num_classes = 10
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)


In [32]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [33]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train2, y_train2) = extract_test_samples('digits')

In [34]:
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [35]:

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [36]:
# create complete model
model = Sequential(feature_layers + classification_layers)


In [37]:
# train model for 5-digit classification [0..4]
train_model(model,
            (x_train, y_train),
            (x_test, y_test), num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/5
469/469 [==============================] - 90s 190ms/step - loss: 2.3073 - accuracy: 0.0979 - val_loss: 2.2673 - val_accuracy: 0.1538
Epoch 2/5
469/469 [==============================] - 89s 190ms/step - loss: 2.2673 - accuracy: 0.1541 - val_loss: 2.2218 - val_accuracy: 0.3259
Epoch 3/5
469/469 [==============================] - 89s 190ms/step - loss: 2.2238 - accuracy: 0.2282 - val_loss: 2.1667 - val_accuracy: 0.4266
Epoch 4/5
469/469 [==============================] - 90s 191ms/step - loss: 2.1747 - accuracy: 0.2957 - val_loss: 2.0977 - val_accuracy: 0.4844
Epoch 5/5
469/469 [==============================] - 90s 191ms/step - loss: 2.1088 - accuracy: 0.3555 - val_loss: 2.0107 - val_accuracy: 0.5316
Training time: 0:07:27.353390
Test score: 2.010667324066162
Test accuracy: 0.5315999984741211


In [38]:

# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)